In [1]:
import torch   # torch: PyTorch의 핵심 패키지로 텐서 연산 등을 제공.
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms #torchvision: PyTorch에서 컴퓨터 비전용 데이터셋, 모델 등을 제공.


# 데이터 전처리 설정
transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 텐서로 변환 transforms.ToTensor(): PIL 이미지나 NumPy 배열을 PyTorch 텐서로 변환.
    transforms.Normalize((0.1307,), (0.3081,))  # 평균과 표준편차로 정규화  transforms.ToTensor(): PIL 이미지나 NumPy 배열을 PyTorch 텐서로 변환.
])

# 데이터셋 로드
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

# datasets.MNIST: MNIST 데이터셋을 로드.
# train=True: 훈련 데이터 로드.
# train=False: 테스트 데이터 로드.
# download=True: 데이터셋이 없으면 다운로드.

# 데이터 로더 설정
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

# DataLoader: 데이터셋을 배치(batch) 단위로 분리하고, 반복 가능(iterable)하게 해줌.
# batch_size: 한 번에 처리할 데이터 샘플 수.
# shuffle: 데이터를 무작위로 섞을지 여부.


# CNN 모델 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 합성곱 층 정의 nn.Conv2d: 2D 합성곱 층을 정의, 입력 채널, 출력 채널, 커널 크기, 스트라이드 순서.
        self.conv1 = nn.Conv2d(1, 32, 3, 1)  # 입력 채널:1, 출력 채널:32, 커널 크기:3x3, 스트라이드:1, nn.MaxPool2d: 2D 맥스 풀링 층을 정의.
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        # 풀링 층 정의
        self.pool = nn.MaxPool2d(2)
        # 전결합 층 정의
        self.fc1 = nn.Linear(9216, 128)  # 9216 = 64채널 * 12 * 12 픽셀
        self.fc2 = nn.Linear(128, 10)    # 출력 클래스 수:10

    def forward(self, x):
        # 합성곱 및 활성화 함수 적용
        x = F.relu(self.conv1(x))  # [batch_size, 32, 26, 26]
        x = F.relu(self.conv2(x))  # [batch_size, 64, 24, 24]
        # 풀링 적용
        x = self.pool(x)           # [batch_size, 64, 12, 12]
        # 텐서 펼치기
        x = x.view(-1, 9216)       # [batch_size, 9216]
        # 전결합 층 및 활성화 함수 적용, x.view(): 텐서의 형태를 변경. -1: 배치 크기에 따라 자동 결정.
        x = F.relu(self.fc1(x))    # [batch_size, 128]
        x = self.fc2(x)            # [batch_size, 10]
        # 출력
        return x


# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train(model, device, train_loader, optimizer, epoch):
    model.train()  # 모델을 학습 모드로 전환, model.train(): 드롭아웃, 배치 정규화 등 학습 시에만 사용되는 기능 활성화
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # 데이터와 레이블을 장치에 할당
        optimizer.zero_grad()  # 기울기 초기화, , model.train(): 드롭아웃, 배치 정규화 등 학습 시에만 사용되는 기능 활성화
        output = model(data)  # 모델에 입력 데이터 전달하여 예측값 계산
        loss = criterion(output, target)  # 손실 계산
        loss.backward()  # 역전파 단계, loss.backward(): 역전파를 통해 기울기 계산.
        optimizer.step()  # 가중치 업데이트, loss.backward(): 역전파를 통해 기울기 계산.

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')


# 평가 함수
def test(model, device, test_loader):
    model.eval()  # 모델을 평가 모드로 전환, model.eval(): 드롭아웃, 배치 정규화 등 평가 시에만 사용되는 기능 비활성화.
    test_loss = 0
    correct = 0
    with torch.no_grad():  # 기울기 계산 비활성화, torch.no_grad(): 기울기 계산을 하지 않아 메모리 사용량 감소 및 속도 향상.
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # 데이터와 레이블을 장치에 할당
            output = model(data)
            test_loss += criterion(output, target).item() * data.size(0)  # 배치 손실 합산
            pred = output.argmax(dim=1, keepdim=True)  # 가장 높은 값을 가진 클래스 선택, output.argmax(dim=1): 각 샘플에 대한 예측 클래스 반환.
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)  # 평균 손실 계산
    accuracy = 100. * correct / len(test_loader.dataset)  # 정확도 계산

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}'
        f' ({accuracy:.2f}%)\n')

# 학습 및 평가 실행
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU 사용 여부 결정, torch.device: GPU를 사용할 수 있으면 사용하고, 아니면 CPU 사용
model.to(device)  # 모델을 장치에 할당, model.to(device): 모델의 파라미터를 장치에 할당.

num_epochs = 5  # 학습 에포크 수, num_epochs: 전체 데이터셋을 몇 번 반복 학습할지 결정.

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)



Train Epoch: 1 [0/60000 (0%)]	Loss: 2.287862
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.201407
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.069766
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.045472
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.161284
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.040828
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.053453
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.090162
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.007213
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.021727

Test set: Average loss: 0.0384, Accuracy: 9871/10000 (98.71%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.028946
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.043339
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.004161
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.019661
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.005005
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.020414
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.028937
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.022560
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.01532